In [1]:
import xarray as xr
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import stats

In [3]:
model = 'cesm2'
realisation = 'r1i1p1f1'
tas = xr.open_dataset('CMIP/tas_'+model+'_180.nc')
tas = tas.sel(lat = slice(-56,84))

obs1 = xr.open_dataset('CMIP/Tair_merge.nc')
obs1 = obs1.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))

#import CESM data for grid reference
elev_sub1 = xr.open_dataset('elev/elev_subnewtas_cesm2.nc')
#elev_sub1 = elev_sub1.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))

elev_grid_bl = xr.open_dataset('elev/elev_gridnewtas_cesm2.nc')
#elev_grid_bl = elev_grid_bl.reindex(lat=list(reversed(elev_grid_bl.lat)))

remap = 'mapped'

tasmapped = xr.open_dataset('review_files/'+remap+'_tas_' + model + '_new.nc')
tasmapped = tasmapped.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))
tasunmapped = xr.open_dataset('CMIP/mnLut_'+ model +'_'+realisation+'_obs.nc')
tasunmapped = tasunmapped.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))


In [9]:
#determine the number of lon and lat at GCM level inside study area
reglon = len(tas.lon.values)
reglat = len(tas.lat.values)

#initialize lapse rate
lapse = 0*obs1.Tair
reg = 0*obs1.Tair

for j in range(0,reglon-1,1):
    for k in range (0,reglat-1,1):
        #slice the data into 1 grid size of GCM model
        lonslice = slice(tas.lon.values[j],tas.lon.values[j+1])
        latslice = slice(tas.lat.values[k+1],tas.lat.values[k])
            
        obslice = obs1.sel(lon = lonslice, lat = latslice)
        elevslice = elev_sub1.sel(lon = lonslice, lat =latslice)
        lapslice = lapse.sel(lon = lonslice, lat = latslice)
        #regslice = reg.sel(lon = lonslice, lat = latslice)
        
        #stack dataset
        stacked_obs = obslice.Tair.stack(z = ("lat","lon"))
        stacked_elev = elevslice.Band1.stack(z = ("lat","lon")).values
        stacked_lapse = lapslice.stack(z = ("lat","lon"))
        #stacked_reg  = regslice.stack(z = ("lat","lon"))
        
        # apply regression between observation and elevation for each timestep and determine slope
        for l in range(144):
            stacked_obs1 = stacked_obs.isel(time =l).values
            mask = ~np.isnan(stacked_elev) & ~np.isnan(stacked_obs1)
            try:
                slope, intercept, r_value, p_value, std_err = stats.linregress(stacked_elev[mask],stacked_obs1[mask])
                #rsq = r_value**2
            except:
                slope = np.nan
                #r_value = np.nan
            stacked_lapse.isel(time = l).values.fill(slope)
            #stacked_reg.isel(time = l).values.fill(rsq)
            
        #unstacked lapse data for lon lat that is filled with the slope from regression
        unstacked_lapse = stacked_lapse.unstack("z")
        #unstacked_reg = stacked_reg.unstack("z")
        lapse.loc[dict(lon = lonslice, lat = latslice)] = unstacked_lapse
        #reg.loc[dict(lon = lonslice, lat = latslice)] = unstacked_reg

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:175: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:189: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:192: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [11]:

#now define the correction by multiplying lapse rate with elevation diference
elev_difer = elev_sub1.Band1 - elev_grid_bl.Band1
lapse=lapse.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))
correction = elev_difer * lapse




In [16]:
tascor = tasmapped.tas.copy(deep =True)
tascor1 = tasunmapped.tasLut.copy(deep = True)


for m in range(len(tasmapped.time.values)):
        tascor[m,:,:] = tasmapped.tas.isel(time = m) + correction.isel(time = m)
        tascor1[m,:,:] = tasunmapped.tasLut.isel(time = m) + correction.isel(time =m)
        tascor[m,:,:] = tascor.isel(time = m).where(obs1.Tair.isel(time = m) >0)
        tascor1[m,:,:] = tascor1.isel(time = m).where(obs1.Tair.isel(time = m) >0)

#    rsq_list[i] = reg.mean()


In [29]:
tascor=tascor.to_dataset()
tascor1=tascor1.to_dataset()
tascor.to_netcdf('review_files/'+remap+'_tascor_'+ model + '_2new.nc')
tascor1.to_netcdf('review_files/mnlut_tascor_' + model + '_2new.nc')

### Do the same for UKESM

In [3]:
#Do the same for CESM
model = 'ukesm'
realisation = 'r1i1p1f2'
tas = xr.open_dataset('CMIP/tas_'+model+'_180.nc')
tas = tas.sel(lat = slice(-56,84))

obs = xr.open_dataset('CMIP/Tair_merge.nc')
obs = obs.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))

#import CESM data for grid reference
elev_sub = xr.open_dataset('elev/elev_sub_uk.nc')

elev_grid = xr.open_dataset('elev/elev_gridnew_ukesm.nc')

remap = 'mapped'

tasmapped = xr.open_dataset('review_files/'+remap+'_tas_' + model + '_new.nc')
tasmapped = tasmapped.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))
tasunmapped = xr.open_dataset('CMIP/mnLut_'+ model +'_'+realisation+'_obs.nc')
tasunmapped = tasunmapped.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))


In [4]:
#determine the number of lon and lat at GCM level inside study area
reglon = len(tas.lon.values)
reglat = len(tas.lat.values)

#initialize lapse rate
lapse = 0*obs.Tair
reg = 0*obs.Tair

for j in range(0,reglon-1,1):
    for k in range (0,reglat-1,1):
        #slice the data into 1 grid size of GCM model
        lonslice = slice(tas.lon.values[j],tas.lon.values[j+1])
        latslice = slice(tas.lat.values[k+1],tas.lat.values[k])
            
        obslice = obs.sel(lon = lonslice, lat = latslice)
        elevslice = elev_sub.sel(lon = lonslice, lat =latslice)
        lapslice = lapse.sel(lon = lonslice, lat = latslice)
        #regslice = reg.sel(lon = lonslice, lat = latslice)
        
        #stack dataset
        stacked_obs = obslice.Tair.stack(z = ("lat","lon"))
        stacked_elev = elevslice.Band1.stack(z = ("lat","lon")).values
        stacked_lapse = lapslice.stack(z = ("lat","lon"))
        #stacked_reg  = regslice.stack(z = ("lat","lon"))
        
        # apply regression between observation and elevation for each timestep and determine slope
        for l in range(144):
            stacked_obs1 = stacked_obs.isel(time =l).values
            mask = ~np.isnan(stacked_elev) & ~np.isnan(stacked_obs1)
            try:
                slope, intercept, r_value, p_value, std_err = stats.linregress(stacked_elev[mask],stacked_obs1[mask])
                #rsq = r_value**2
            except:
                slope = np.nan
                #r_value = np.nan
            stacked_lapse.isel(time = l).values.fill(slope)
            #stacked_reg.isel(time = l).values.fill(rsq)
            
        #unstacked lapse data for lon lat that is filled with the slope from regression
        unstacked_lapse = stacked_lapse.unstack("z")
        #unstacked_reg = stacked_reg.unstack("z")
        lapse.loc[dict(lon = lonslice, lat = latslice)] = unstacked_lapse
        #reg.loc[dict(lon = lonslice, lat = latslice)] = unstacked_reg

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:175: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:189: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:192: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [6]:
#elev_grid = xr.open_dataset('elev/elev_small_' + model + '.nc')
#now define the correction by multiplying lapse rate with elevation diference
elev_difer = elev_sub.Band1 - elev_grid.Band1
lapse=lapse.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))
correction = elev_difer * lapse




In [8]:
tasmapped=tasmapped.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))
tasunmapped=tasunmapped.sel(lat = slice(83.125,-55.625),lon = slice(-179.0625,179.0625))

In [9]:
tascor = tasmapped.tas.copy(deep =True)
tascor1 = tasunmapped.tasLut.copy(deep = True)

for m in range(len(tasmapped.time.values)):
        tascor[m,:,:] = tasmapped.tas.isel(time = m) + correction.isel(time = m)
        tascor1[m,:,:] = tasunmapped.tasLut.isel(time = m) + correction.isel(time =m)
        tascor[m,:,:] = tascor.isel(time = m).where(obs.Tair.isel(time = m) >0)
        tascor1[m,:,:] = tascor1.isel(time = m).where(obs.Tair.isel(time = m) >0)

#    rsq_list[i] = reg.mean()


In [13]:

tascor=tascor.to_dataset()
tascor1=tascor1.to_dataset()
tascor.to_netcdf('review_files/'+remap+'_tascor_'+ model + '_new.nc')
tascor1.to_netcdf('review_files/mnlut_tascor_' + model + '_new.nc')

### For LST

In [9]:

model = 'cesm2'
realisation = 'r1i1p1f1'
tas2 = xr.open_dataset('lstmip/tslsi_'+model+'_180.nc')
tas2 = tas2.sel(lat = slice(-56,84))

obs2 = xr.open_dataset('lstmip/modlst.nc')
obs2 = obs2.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))

#import CESM data for grid reference
elev_sub2= xr.open_dataset('elev/elev_subnewlst_cesm2.nc')
elev_grid2 = xr.open_dataset('elev/elev_gridnewlst_cesm2.nc')

remap = 'mapped'

tasmapped = xr.open_dataset('review_files/'+remap+'_tslsi_' + model + '_new.nc')
tasmapped = tasmapped.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))
tasunmapped = xr.open_dataset('lstmip/mnLut_'+ model +'_obs.nc')
tasunmapped = tasunmapped.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))


In [3]:
#determine the number of lon and lat at GCM level inside study area
reglon = len(tas2.lon.values)
reglat = len(tas2.lat.values)

#initialize lapse rate
lapse = 0*obs2.lst
reg = 0*obs2.lst

for j in range(0,reglon-1,1):
    for k in range (0,reglat-1,1):
        #slice the data into 1 grid size of GCM model
        lonslice = slice(tas2.lon.values[j],tas2.lon.values[j+1])
        latslice = slice(tas2.lat.values[k+1],tas2.lat.values[k])
            
        obslice = obs2.sel(lon = lonslice, lat = latslice)
        elevslice = elev_sub2.sel(lon = lonslice, lat =latslice)
        lapslice = lapse.sel(lon = lonslice, lat = latslice)
        #regslice = reg.sel(lon = lonslice, lat = latslice)
        
        #stack dataset
        stacked_obs = obslice.lst.stack(z = ("lat","lon"))
        stacked_elev = elevslice.Band1.stack(z = ("lat","lon")).values
        stacked_lapse = lapslice.stack(z = ("lat","lon"))
        #stacked_reg  = regslice.stack(z = ("lat","lon"))
        
        # apply regression between observation and elevation for each timestep and determine slope
        for l in range(144):
            stacked_obs1 = stacked_obs.isel(time =l).values
            mask = ~np.isnan(stacked_elev) & ~np.isnan(stacked_obs1)
            try:
                slope, intercept, r_value, p_value, std_err = stats.linregress(stacked_elev[mask],stacked_obs1[mask])
                #rsq = r_value**2
            except:
                slope = np.nan
                #r_value = np.nan
            stacked_lapse.isel(time = l).values.fill(slope)
            #stacked_reg.isel(time = l).values.fill(rsq)
            
        #unstacked lapse data for lon lat that is filled with the slope from regression
        unstacked_lapse = stacked_lapse.unstack("z")
        #unstacked_reg = stacked_reg.unstack("z")
        lapse.loc[dict(lon = lonslice, lat = latslice)] = unstacked_lapse
        #reg.loc[dict(lon = lonslice, lat = latslice)] = unstacked_reg

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:175: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:189: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:192: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [4]:

#now define the correction by multiplying lapse rate with elevation diference
elev_difer = elev_sub2.Band1 - elev_grid2.Band1
lapse = lapse.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))
correction = elev_difer * lapse




In [12]:
tascor = tasmapped.tas.copy(deep =True)
tascor1 = tasunmapped.tslsiLut.copy(deep = True)

for m in range(len(tasmapped.time.values)):
        tascor[m,:,:] = tasmapped.tas.isel(time = m) + correction.isel(time = m)
        tascor1[m,:,:] = tasunmapped.tslsiLut.isel(time = m) + correction.isel(time =m)
        tascor[m,:,:] = tascor.isel(time = m).where(obs2.lst.isel(time = m) >0)
        tascor1[m,:,:] = tascor1.isel(time = m).where(obs2.lst.isel(time = m) >0)

#    rsq_list[i] = reg.mean()


In [16]:
tascor.to_netcdf('review_files/mapped_tslsicor_'+ model + '_2new.nc')
tascor1.to_netcdf('review_files/mnlut_tslsicor_' + model + '_2new.nc')